# Imports

In [1]:

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import warnings
import data_func.read_data as read_data
warnings.filterwarnings("ignore", category=FutureWarning, module="xgboost")
warnings.filterwarnings("ignore", category=FutureWarning, message="is_sparse is deprecated")


# Load datasets

In [2]:
dataframes = read_data.get_training_data()
X_frames_train = dataframes[0]
Y_frames_train = dataframes[1]
X_frames_test = read_data.get_test_data()


In [3]:
XTRA = X_frames_train[0]
XTRB = X_frames_train[1]
XTRC = X_frames_train[2]

YA = Y_frames_train[0]
YB = Y_frames_train[1]
YC = Y_frames_train[2]

drop 0 since we want to match on the "whole" hour.

In [4]:
for i in range(len(X_frames_test)):
    X_frames_test[i] = X_frames_test[i].drop(columns=["date_calc"])

# Data clean up

In [5]:
# making shure that target values line up with x_values
import data_func.aggregation as data_agg

categorical_col = ['dew_or_rime:idx', 'precip_type_5min:idx']

def aggregate_correct_x(x: pd.DataFrame) -> pd.DataFrame:
   categorical = x[["date_forecast"] + categorical_col]
   mean = x.drop(columns=categorical_col)

   categorical = data_agg.gen_agg(categorical, data_agg.stocastic_median)
   mean = data_agg.gen_agg(mean, "mean")

   return pd.merge(categorical, mean, on="date_forecast")

def data_allign(x_train, y_train):

  y_train.dropna(inplace=True)
  x_train = aggregate_correct_x(x_train)
  combined_data = pd.merge(x_train, y_train, left_on='date_forecast', right_on='time')
  y_train = combined_data['pv_measurement']

  if 'time' and 'pv_measurement' in combined_data.columns:
    combined_data.drop(columns=['time', 'pv_measurement'], inplace=True)
    
  return combined_data, y_train



X_train = [XTRA, XTRB, XTRC]
Y_train = [YA, YB, YC]

for i in range(len(X_train)):
    X_train[i], Y_train[i] = data_allign(X_train[i], Y_train[i])

for j in range(len(X_frames_test)):
    X_frames_test[j] = aggregate_correct_x(X_frames_test[j])




In [6]:
#Verify length matches
for x in range(len(X_train)):
    print("x,y: ", len(X_train[x]), len(Y_train[x]))



x,y:  29667 29667
x,y:  29218 29218
x,y:  23141 23141


# Feature engineering

In [7]:
import data_func.timeseasonality as DTS
import data_func.one_hot_encoding as OHE
for i in range(len(X_train)):
    X_train[i] = DTS.append_seasonal_columns(X_train[i])
    X_train[i].drop(columns=['date_forecast'], inplace=True)

for i in range(len(X_frames_test)):
    X_frames_test[i] = DTS.append_seasonal_columns(X_frames_test[i])
    X_frames_test[i].drop(columns=['date_forecast'], inplace=True)

# THIS SECTION CAN ONLY WORK IF THE PREVIOUS AGGREGATION IS DONE INDIVIDUALLY FOR CATEGORICAL DATA
# import data_func.one_hot_encoding as OHE

# for i in range(len(X_train)):
#      X_train[i] = OHE.one_hot_encode(X_train[i], ['dew_or_rime:idx', 'precip_type_5min:idx'])

# for i in range(len(X_frames_test)):
#      X_frames_test[i] = OHE.one_hot_encode(X_frames_test[i], ['dew_or_rime:idx', 'precip_type_5min:idx'])
#      X_frames_test[i]["dew_or_rime:idx_-1"] = 0 
#      X_frames_test[i]["precip_type_5min:idx_2"] = 0 
#      X_frames_test[i]["precip_type_5min:idx_3"] = 0 



In [8]:
X_train[0].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
dew_or_rime:idx,29667.0,0.016854,0.171208,-1.000000,0.000000,0.000000e+00,0.000000,1.000000e+00
precip_type_5min:idx,29667.0,0.068123,0.323120,0.000000,0.000000,0.000000e+00,0.000000,5.000000e+00
absolute_humidity_2m:gm3,29667.0,6.379581,2.612027,0.700000,4.400000,5.975000e+00,8.100000,1.605000e+01
air_density_2m:kgm3,29667.0,1.252307,0.034827,1.145000,1.229000,1.251000e+00,1.274000,1.426250e+00
ceiling_height_agl:m,24681.0,3043.045898,2555.343994,27.849998,1183.925049,2.080000e+03,4255.549805,1.228565e+04
clear_sky_energy_1h:J,29667.0,565528.562500,825693.562500,0.000000,0.000000,4.782730e+04,959101.687500,2.988628e+06
clear_sky_rad:W,29667.0,157.091171,229.956268,0.000000,0.000000,1.325000e+01,273.575012,8.351000e+02
cloud_base_agl:m,28055.0,1746.576660,1822.560547,27.900000,598.125000,1.160900e+03,2088.106201,1.167362e+04
dew_point_2m:K,29667.0,276.139557,6.374989,251.074997,271.875000,2.760500e+02,280.924988,2.919500e+02
diffuse_rad:W,29667.0,42.953587,61.441681,0.000000,0.000000,6.162500e+00,72.262501,3.322750e+02


In [9]:
X_frames_test[0].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
dew_or_rime:idx,720.0,2.916667e-02,1.683905e-01,0.000000,0.000000,0.000000e+00,0.000000e+00,1.000000e+00
precip_type_5min:idx,720.0,6.388889e-02,2.447249e-01,0.000000,0.000000,0.000000e+00,0.000000e+00,1.000000e+00
absolute_humidity_2m:gm3,720.0,8.205903e+00,2.178425e+00,3.200000,6.693750,8.050000e+00,9.981250e+00,1.367500e+01
air_density_2m:kgm3,720.0,1.233150e+00,3.210283e-02,1.151000,1.209000,1.238625e+00,1.260000e+00,1.299750e+00
ceiling_height_agl:m,532.0,3.096333e+03,2.945732e+03,53.299999,953.000000,1.712700e+03,4.097362e+03,1.143078e+04
clear_sky_energy_1h:J,720.0,1.227651e+06,1.101479e+06,0.000000,48799.869141,9.337900e+05,2.276145e+06,2.987530e+06
clear_sky_rad:W,720.0,3.410140e+02,3.069119e+02,0.000000,17.950000,2.995750e+02,6.617375e+02,8.349500e+02
cloud_base_agl:m,650.0,1.891609e+03,2.128155e+03,30.225000,509.556274,1.035875e+03,2.437175e+03,1.125670e+04
dew_point_2m:K,720.0,2.808002e+02,4.339592e+00,268.100006,278.024994,2.810500e+02,2.843812e+02,2.895500e+02
diffuse_rad:W,720.0,8.490232e+01,7.859020e+01,0.000000,8.087500,7.530000e+01,1.342313e+02,3.075000e+02


## Dropping data

In [12]:
drop_col = ["snow_density:kgm3", 
            "pressure_100m:hPa", 
            "pressure_50m:hPa", 
            "snow_depth:cm", 
            "ceiling_height_agl:m", 
            "fresh_snow_24h:cm", 
            "fresh_snow_3h:cm"]

for i in range(len(X_train)):
    X_train[i] = X_train[i].drop(columns = drop_col )
    X_frames_test[i] = X_frames_test[i].drop(columns= drop_col)


# Training the model

In [13]:
x_train_a, x_val_a, y_train_a, y_val_a = train_test_split(X_train[0], Y_train[0], test_size=0.17, random_state=None)
x_train_b, x_val_b, y_train_b, y_val_b = train_test_split(X_train[1], Y_train[1], test_size=0.17, random_state=None)
x_train_c, x_val_c, y_train_c, y_val_c = train_test_split(X_train[2], Y_train[2], test_size=0.17, random_state=None)

In [14]:
# Use params from hyperparameter tuning using optuna
params_a = {'random_state': 42, 'n_estimators': 980, 'max_depth': 9, 'learning_rate': 0.029035565559484028, 'subsample': 0.8393121619033767, 'colsample_bytree': 0.7589542758688459}
params_b = {'random_state': 42, 'n_estimators': 955, 'max_depth': 9, 'learning_rate': 0.02949625834198986, 'subsample': 0.8030196155828968, 'colsample_bytree': 0.72518389089994}
params_c = {'random_state': 42, 'n_estimators': 727, 'max_depth': 8, 'learning_rate': 0.08609213174337473, 'subsample': 0.8107057409889747, 'colsample_bytree': 0.8763563332327975}
model_a = xgb.XGBRegressor(**params_a)
model_b = xgb.XGBRegressor(**params_b)
model_c = xgb.XGBRegressor(**params_c)



In [15]:

model_a.fit(x_train_a, y_train_a)
model_b.fit(x_train_b, y_train_b)
model_c.fit(x_train_c, y_train_c)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8763563332327975, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.08609213174337473, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=727, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

# Make predictions

In [17]:
model_a.fit(X_train[0], Y_train[0])
model_b.fit(X_train[1], Y_train[1])
model_c.fit(X_train[2], Y_train[2])

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8763563332327975, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.08609213174337473, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=727, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [18]:
# Do some more stuff
y_pred_a = model_a.predict(X_frames_test[0])
y_pred_b = model_b.predict(X_frames_test[1])
y_pred_c = model_c.predict(X_frames_test[2])


# Evaluate prediction

In [16]:

# Evaluate the model based on the validation data

mse_a = mean_squared_error(y_val_a, model_a.predict(x_val_a))
print("MSE for A: ", mse_a)
mse_b = mean_squared_error(y_val_b, model_b.predict(x_val_b))
print("MSE for B: ", mse_b)
mse_c = mean_squared_error(y_val_c, model_c.predict(x_val_c))
print("MSE for C: ", mse_c)
print("Mean MSE: ", (mse_a + mse_b + mse_c) / 3)

# Evaluate the predictions

score_a = model_a.score(x_val_a, y_val_a)
score_b = model_b.score(x_val_b, y_val_b)
score_c = model_c.score(x_val_c, y_val_c)

print("Score A: ", score_a)
print("Score B: ", score_b)
print("Score C: ", score_c)
print('')

# Get feature importance scores
models = [(model_a, 'A'), (model_b, 'B'), (model_c, 'C')]
for model in models:

    feature_importance_scores = model[0].feature_importances_

# Create a DataFrame to associate features with their importance scores
    feature_importance_df1 = pd.DataFrame({'Feature': x_train_a.columns, 'Importance': feature_importance_scores})

# Sort features by importance in descending order
    feature_importance_df1 = feature_importance_df1.sort_values(by='Importance', ascending=False)

# Print or visualize the feature importance scores
    
    print(f'Model {model[1]}')
    print(feature_importance_df1.head(10))
    print('')


MSE for A:  153312.4186017793
MSE for B:  4016.0402458779813
MSE for C:  2535.913950434679
Mean MSE:  53288.12426603065
Score A:  0.8946682377882834
Score B:  0.9022832738988866
Score C:  0.9154874520770043

Model A
             Feature  Importance
10      direct_rad:W    0.393947
11   direct_rad_1h:J    0.126903
8      diffuse_rad:W    0.049902
5    clear_sky_rad:W    0.042737
18  is_in_shadow:idx    0.027637
20    precip_5min:mm    0.027032
41      cosinus_year    0.025505
29   sun_elevation:d    0.024073
28     sun_azimuth:d    0.024016
38         sinus_day    0.019216

Model B
              Feature  Importance
10       direct_rad:W    0.250389
5     clear_sky_rad:W    0.122218
18   is_in_shadow:idx    0.103694
29    sun_elevation:d    0.098762
17         is_day:idx    0.050124
16   fresh_snow_6h:cm    0.045439
14  fresh_snow_12h:cm    0.034243
41       cosinus_year    0.026628
11    direct_rad_1h:J    0.024638
8       diffuse_rad:W    0.021780

Model C
                  Feature  Im

# Create submission

In [19]:
y_pred = np.concatenate((y_pred_a, y_pred_b, y_pred_c), axis=0)

In [20]:
y_test_pred = y_pred

test = pd.read_csv('../data/test.csv')
test['prediction'] = y_test_pred
sample_submission = pd.read_csv('../data/sample_submission.csv')
submission = sample_submission[['id']].merge(test[['id', 'prediction']], on='id', how='left')
submission.to_csv('submission_xg_y.csv', index=False)